## The Dog v Cat Problem using VGG19
The pretrained VGG19 was loaded and connected with a trainable finaly fully connected layer

The script vgg19.py contains the VGG19 class and the loader function.  
Running
```python
vggnet = VGG19(VGG_LINK, VGG_FILE)
```
downloads the VGG19 parameters if not already present in the working directory.  
The 
```python
vggnet.build()
```
function builds the VGG19 graph with the downloaded parameters

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import tensorflow as tf

from tflearn.data_utils import to_categorical
from utils import load_data
from utils import vgg19
from utils import batchify

In [2]:
VGG_LINK = 'http://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat'
VGG_FILE = 'imagenet-vgg-verydeep-19.mat'
VGG_MEAN = [103.939, 116.779, 123.68]
BATCH_SIZE = 10
IMAGE_SIZE = (224, 224)

In [3]:
X_train, Y_train, X_test = load_data('G:/Python/', IMAGE_SIZE)

In [4]:
vggnet = VGG19(VGG_LINK, VGG_FILE)

VGGNet raw_file present


In [5]:
X = tf.placeholder(tf.float32, shape=(BATCH_SIZE, *IMAGE_SIZE, 3), name='X')
Y = tf.placeholder(tf.float32, shape=(None, 2), name='Y')

In [6]:
vgg_graph = vggnet.build(X)

In [7]:
def two_label_layer(input_layer):
    shape = input_layer.get_shape().as_list()
    dim = 1
    for d in shape[1:]:
        dim *= d
    x = tf.reshape(input_layer, [-1, dim])

    # the '1' is the number of labels in the dog-cat problem
    weights = tf.Variable(tf.random_normal([dim, 2]))
    biases = tf.Variable(tf.random_normal([2]))

    output_layer = tf.nn.bias_add(tf.matmul(x, weights), biases)
    return(output_layer)

In [8]:
output_layer = two_label_layer(vgg_graph['full8_1'])

In [10]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=output_layer, labels=Y))

In [11]:
optimizer = tf.train.AdamOptimizer(1.0).minimize(loss)

In [12]:
init = tf.global_variables_initializer()

In [13]:
with tf.Session() as sess:
    sess.run(init)
    number_batches = int(len(X_train)/BATCH_SIZE)
    training_loss_list = []
    for epoch in range(1):
        for batch in batchify(X_train, Y_train, BATCH_SIZE):
            X_batch, Y_batch = batch
            _, training_loss_value = sess.run([optimizer, loss], feed_dict={X: X_batch, Y: Y_batch})
            training_loss_list.append(training_loss_value)

In [14]:
training_loss_list

[135.36212, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]